# CASE STUDY:DATA EXPLORATORY ANALYSIS AND HYPOTHESIS TESTING FOR INSURANCE CLAIMS DATA

## ANALYTICS IN INSURANCE CLAIMS

## BUSINESS PROBLEM:

## DATA AVAILABLE -

    1. Import claims_data.csv and cust_data.csv which is provided to you and combine the two datasets appropriately to create a 360-degree view of the data. Use the same for the subsequent questions.
    2. Perform a data audit for the datatypes and find out if there are any mismatch within the current datatypes of the columns and their business significance.
    3. Convert the column claim_amount to numeric. Use the appropriate modules/attributes to remove the $ sign$.
    
    4. Of all the injury claims, some of them have gone unreported with the police. Create an alert flag (1,0) for all such claims.
    5. One customer can claim for insurance more than once and in each claim, multiple categories of claims can be involved. However, customer ID should remain unique. Retain the most recent observation and delete any duplicated records in the data based on the customer ID column.
    6. Check for missing values and impute the missing values with an appropriate value. (mean for continuous and mode for categorical)
    7. Calculate the age of customers in years. Based on the age, categorize the customers according to the below criteria
        Children < 18
        Youth 18-30
        Adult 30-60
        Senior > 60
    8. What is the average amount claimed by the customers from various segments?
    9. What is the total claim amount based on incident cause for all the claims that have been done at least 20 days prior to 1st of October, 2018.
    10. How many adults from TX, DE and AK claimed insurance for driver related issues and causes?
    11. Draw a pie chart between the aggregated value of claim amount based on gender and segment. Represent the claim amount as a percentage on the pie chart.
    12. Among males and females, which gender had claimed the most for any type of driver related issues? E.g. This metric can be compared using a bar chart
    13. Which age group had the maximum fraudulent policy claims? Visualize it on a bar chart.
    14. Visualize the monthly trend of the total amount that has been claimed by the customers. Ensure that on the “month” axis, the month is in a chronological order not alphabetical order.
    15. What is the average claim amount for gender and age categories and suitably represent the above using a facetted bar chart, one facet that represents fraudulent claims and the other for non-fraudulent claims.
    Based on the conclusions from exploratory analysis as well as suitable statistical tests, answer the below questions. Please include a detailed write-up on the parameters taken into consideration, the Hypothesis testing steps, conclusion from the p-values and the business implications of the statements.
    16. Is there any similarity in the amount claimed by males and females?
    17. Is there any relationship between age category and segment?
    18. The current year has shown a significant rise in claim amounts as
    compared to 2016-17 fiscal average which was $10,000.
    19. Is there any difference between age groups and insurance claims?
    20. Is there any relationship between total number of policy claims and the
    claimed amount?
    
---
---

## Import Libraries

In [1]:
# Handle table-like data and matrices
import pandas as pd
import numpy as np
from numpy import percentile
from datetime import date
import datetime
from datetime import timedelta
import scipy.stats as stats
import seaborn as sns
from scipy import stats

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
import plotly
import plotly.express as px
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline()
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configure visualisations
%matplotlib inline

## Load Data

In [2]:
claim_data= pd.read_csv('claims.csv')
claim_data

,claim_id,customer_id,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent
0,54004764,21868593,Driver error,11/27/2017,Auto,No,Material only,$2980,1.0,No
1,33985796,75740424,Crime,10/03/2018,Home,Unknown,Material only,$2980,3.0,No
2,53522022,30308357,Other driver error,02/02/2018,Auto,No,Material only,$3369.5,1.0,Yes
3,13015401,47830476,Natural causes,06/17/2018,Auto,No,Material only,$1680,1.0,No
4,22890252,19269962,Crime,01/13/2018,Auto,No,Material only,$2680,1.0,No
...,...,...,...,...,...,...,...,...,...,...
1095,97727122,35951012,Other driver error,06/11/2017,Auto,No,Material only,$3059,4.0,Yes
1096,10247193,14818669,Natural causes,03/14/2018,Auto,No,Material only,$1520,2.0,No
1097,79807493,85322831,Other driver error,02/09/2018,Auto,No,Material and injury,$23575,2.0,Yes
1098,69299345,73449366,Other causes,03/21/2018,Auto,Unknown,Material and injury,$25120,1.0,No


In [3]:
cust_data= pd.read_csv('cust_demographics.csv')
cust_data

,CUST_ID,gender,DateOfBirth,State,Contact,Segment
0,21868593,Female,12-Jan-79,VT,789-916-8172,Platinum
1,75740424,Female,13-Jan-70,ME,265-543-1264,Silver
2,30308357,Female,11-Mar-84,TN,798-631-4758,Silver
3,47830476,Female,01-May-86,MA,413-187-7945,Silver
4,19269962,Male,13-May-77,NV,956-871-8691,Gold
...,...,...,...,...,...,...
1080,79539873,Female,15-Mar-81,ND,459-425-4319,Platinum
1081,42364152,Female,07-Jul-96,ID,529-462-1635,Silver
1082,19888166,Male,11-Apr-90,WI,712-651-9613,Gold
1083,11256802,Female,22-Oct-64,LA,469-345-5617,Silver


In [4]:
claim_data.rename(columns={"customer_id":"CUST_ID"},inplace=True)

In [5]:
claim_data['claim_id']=claim_data['claim_id'].astype('object')
claim_data["CUST_ID"] = claim_data["CUST_ID"].astype('object')
claim_data["claim_date"] = claim_data["claim_date"].astype('datetime64[ns]')
claim_data['claim_amount']=claim_data['claim_amount'].str.strip('$').astype(float)
claim_data

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No
1,33985796,75740424,Crime,2018-10-03,Home,Unknown,Material only,2980.0,3.0,No
2,53522022,30308357,Other driver error,2018-02-02,Auto,No,Material only,3369.5,1.0,Yes
3,13015401,47830476,Natural causes,2018-06-17,Auto,No,Material only,1680.0,1.0,No
4,22890252,19269962,Crime,2018-01-13,Auto,No,Material only,2680.0,1.0,No
...,...,...,...,...,...,...,...,...,...,...
1095,97727122,35951012,Other driver error,2017-06-11,Auto,No,Material only,3059.0,4.0,Yes
1096,10247193,14818669,Natural causes,2018-03-14,Auto,No,Material only,1520.0,2.0,No
1097,79807493,85322831,Other driver error,2018-02-09,Auto,No,Material and injury,23575.0,2.0,Yes
1098,69299345,73449366,Other causes,2018-03-21,Auto,Unknown,Material and injury,25120.0,1.0,No


In [6]:
claim_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   claim_id             1100 non-null   object        
 1   CUST_ID              1100 non-null   object        
 2   incident_cause       1100 non-null   object        
 3   claim_date           1100 non-null   datetime64[ns]
 4   claim_area           1100 non-null   object        
 5   police_report        1100 non-null   object        
 6   claim_type           1100 non-null   object        
 7   claim_amount         1035 non-null   float64       
 8   total_policy_claims  1090 non-null   float64       
 9   fraudulent           1100 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 86.1+ KB


In [7]:
cust_data["CUST_ID"] = cust_data["CUST_ID"].astype('object')
cust_data["DateOfBirth"] = cust_data["DateOfBirth"].astype('datetime64[ns]')
cust_data

,CUST_ID,gender,DateOfBirth,State,Contact,Segment
0,21868593,Female,1979-01-12,VT,789-916-8172,Platinum
1,75740424,Female,2070-01-13,ME,265-543-1264,Silver
2,30308357,Female,1984-03-11,TN,798-631-4758,Silver
3,47830476,Female,1986-05-01,MA,413-187-7945,Silver
4,19269962,Male,1977-05-13,NV,956-871-8691,Gold
...,...,...,...,...,...,...
1080,79539873,Female,1981-03-15,ND,459-425-4319,Platinum
1081,42364152,Female,1996-07-07,ID,529-462-1635,Silver
1082,19888166,Male,1990-04-11,WI,712-651-9613,Gold
1083,11256802,Female,2064-10-22,LA,469-345-5617,Silver


In [8]:
cust_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1085 entries, 0 to 1084
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CUST_ID      1085 non-null   object        
 1   gender       1085 non-null   object        
 2   DateOfBirth  1085 non-null   datetime64[ns]
 3   State        1085 non-null   object        
 4   Contact      1085 non-null   object        
 5   Segment      1085 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 51.0+ KB


### 1. Import claims_data.csv and cust_data.csv which is provided to you and combine the two datasets appropriately to create a 360-degree view of the data. Use the same for the subsequent questions.

In [9]:
cust_claim_data= pd.merge(left=claim_data, right=cust_data, on=['CUST_ID'], how='inner')
cust_claim_data

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No,Female,1979-01-12,VT,789-916-8172,Platinum
1,33985796,75740424,Crime,2018-10-03,Home,Unknown,Material only,2980.0,3.0,No,Female,2070-01-13,ME,265-543-1264,Silver
2,53522022,30308357,Other driver error,2018-02-02,Auto,No,Material only,3369.5,1.0,Yes,Female,1984-03-11,TN,798-631-4758,Silver
3,63017412,30308357,Driver error,2018-04-04,Auto,No,Material only,1950.0,6.0,No,Female,1984-03-11,TN,798-631-4758,Silver
4,13015401,47830476,Natural causes,2018-06-17,Auto,No,Material only,1680.0,1.0,No,Female,1986-05-01,MA,413-187-7945,Silver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,62020796,79539873,Other driver error,2018-09-06,Auto,Yes,Material only,1770.0,1.0,No,Female,1981-03-15,ND,459-425-4319,Platinum
1081,63269067,42364152,Other causes,2017-05-09,Home,No,Material only,2800.0,1.0,No,Female,1996-07-07,ID,529-462-1635,Silver
1082,22690595,19888166,Other driver error,2017-05-15,Auto,Yes,Injury only,33080.0,2.0,No,Male,1990-04-11,WI,712-651-9613,Gold
1083,12376894,11256802,Other driver error,2017-04-18,Auto,No,Material only,2840.0,4.0,No,Female,2064-10-22,LA,469-345-5617,Silver


### 2. Perform a data audit for the datatypes and find out if there are any mismatch within the current datatypes of the columns and their business significance.

In [10]:
cust_claim_data.describe(include='all',datetime_is_numeric=True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
claim_id,1085.0,1085.0,54004764.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CUST_ID,1085.0,1078.0,84147585.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
incident_cause,1085,5,Other causes,285,NaN,NaN,NaN,NaN,NaN,NaN,NaN
claim_date,1085,NaN,NaN,NaN,2017-11-26 01:03:42.304147456,2017-01-01 00:00:00,2017-06-04 00:00:00,2017-11-27 00:00:00,2018-05-14 00:00:00,2018-10-30 00:00:00,NaN
claim_area,1085,2,Auto,970,NaN,NaN,NaN,NaN,NaN,NaN,NaN
police_report,1085,3,No,621,NaN,NaN,NaN,NaN,NaN,NaN,NaN
claim_type,1085,3,Material only,653,NaN,NaN,NaN,NaN,NaN,NaN,NaN
claim_amount,1020.0,NaN,NaN,NaN,12467.681373,1000.0,1870.0,2740.0,24260.0,48150.5,13783.249662
total_policy_claims,1075.0,NaN,NaN,NaN,1.586977,1.0,1.0,1.0,2.0,8.0,1.146859
fraudulent,1085,2,No,838,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Convert the column claim_amount to numeric. Use the appropriate modules/attributes to remove the $ sign.

In [11]:
# Done above step done:
#claim_data['claim_amount']=claim_data['claim_amount'].str.strip('$').astype(float)

### 4. Of all the injury claims, some of them have gone unreported with the police. Create an alert flag (1,0) for all such claims.

In [12]:
cust_claim_data['alert_flag']= np.where(cust_claim_data.police_report == 'Unknown', 1, 0)
cust_claim_data

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment,alert_flag
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No,Female,1979-01-12,VT,789-916-8172,Platinum,0
1,33985796,75740424,Crime,2018-10-03,Home,Unknown,Material only,2980.0,3.0,No,Female,2070-01-13,ME,265-543-1264,Silver,1
2,53522022,30308357,Other driver error,2018-02-02,Auto,No,Material only,3369.5,1.0,Yes,Female,1984-03-11,TN,798-631-4758,Silver,0
3,63017412,30308357,Driver error,2018-04-04,Auto,No,Material only,1950.0,6.0,No,Female,1984-03-11,TN,798-631-4758,Silver,0
4,13015401,47830476,Natural causes,2018-06-17,Auto,No,Material only,1680.0,1.0,No,Female,1986-05-01,MA,413-187-7945,Silver,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,62020796,79539873,Other driver error,2018-09-06,Auto,Yes,Material only,1770.0,1.0,No,Female,1981-03-15,ND,459-425-4319,Platinum,0
1081,63269067,42364152,Other causes,2017-05-09,Home,No,Material only,2800.0,1.0,No,Female,1996-07-07,ID,529-462-1635,Silver,0
1082,22690595,19888166,Other driver error,2017-05-15,Auto,Yes,Injury only,33080.0,2.0,No,Male,1990-04-11,WI,712-651-9613,Gold,0
1083,12376894,11256802,Other driver error,2017-04-18,Auto,No,Material only,2840.0,4.0,No,Female,2064-10-22,LA,469-345-5617,Silver,0


### 5. One customer can claim for insurance more than once and in each claim, multiple categories of claims can be involved. However, customer ID should remain unique.
### Retain the most recent observation and delete any duplicated records in the data based on the customer ID column. 

In [13]:
cust_claim_data.drop_duplicates(subset = 'CUST_ID',keep = 'last')

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment,alert_flag
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No,Female,1979-01-12,VT,789-916-8172,Platinum,0
1,33985796,75740424,Crime,2018-10-03,Home,Unknown,Material only,2980.0,3.0,No,Female,2070-01-13,ME,265-543-1264,Silver,1
3,63017412,30308357,Driver error,2018-04-04,Auto,No,Material only,1950.0,6.0,No,Female,1984-03-11,TN,798-631-4758,Silver,0
5,91609100,47830476,Natural causes,2017-04-12,Auto,Unknown,Material only,1440.0,1.0,No,Female,1986-05-01,MA,413-187-7945,Silver,1
7,39219616,19269962,Other causes,2018-09-26,Auto,No,Material only,2240.0,2.0,No,Male,1977-05-13,NV,956-871-8691,Gold,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,62020796,79539873,Other driver error,2018-09-06,Auto,Yes,Material only,1770.0,1.0,No,Female,1981-03-15,ND,459-425-4319,Platinum,0
1081,63269067,42364152,Other causes,2017-05-09,Home,No,Material only,2800.0,1.0,No,Female,1996-07-07,ID,529-462-1635,Silver,0
1082,22690595,19888166,Other driver error,2017-05-15,Auto,Yes,Injury only,33080.0,2.0,No,Male,1990-04-11,WI,712-651-9613,Gold,0
1083,12376894,11256802,Other driver error,2017-04-18,Auto,No,Material only,2840.0,4.0,No,Female,2064-10-22,LA,469-345-5617,Silver,0


### 6. Check for missing values and impute the missing values with an appropriate value. (mean for continuous and mode for categorical)

In [14]:
cust_claim_data.isnull().sum()

claim_id                0
CUST_ID                 0
incident_cause          0
claim_date              0
claim_area              0
police_report           0
claim_type              0
claim_amount           65
total_policy_claims    10
fraudulent              0
gender                  0
DateOfBirth             0
State                   0
Contact                 0
Segment                 0
alert_flag              0
dtype: int64

In [15]:
mod=cust_claim_data['total_policy_claims'].mode()
print(mod)

0    1.0
Name: total_policy_claims, dtype: float64


In [16]:
#Categorical values
cust_claim_data['total_policy_claims'].fillna('1.0', inplace=True)

#Continous values
cust_claim_data['claim_amount'].fillna(cust_claim_data['claim_amount'].mean(), inplace=True)

cust_claim_data

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment,alert_flag
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No,Female,1979-01-12,VT,789-916-8172,Platinum,0
1,33985796,75740424,Crime,2018-10-03,Home,Unknown,Material only,2980.0,3.0,No,Female,2070-01-13,ME,265-543-1264,Silver,1
2,53522022,30308357,Other driver error,2018-02-02,Auto,No,Material only,3369.5,1.0,Yes,Female,1984-03-11,TN,798-631-4758,Silver,0
3,63017412,30308357,Driver error,2018-04-04,Auto,No,Material only,1950.0,6.0,No,Female,1984-03-11,TN,798-631-4758,Silver,0
4,13015401,47830476,Natural causes,2018-06-17,Auto,No,Material only,1680.0,1.0,No,Female,1986-05-01,MA,413-187-7945,Silver,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,62020796,79539873,Other driver error,2018-09-06,Auto,Yes,Material only,1770.0,1.0,No,Female,1981-03-15,ND,459-425-4319,Platinum,0
1081,63269067,42364152,Other causes,2017-05-09,Home,No,Material only,2800.0,1.0,No,Female,1996-07-07,ID,529-462-1635,Silver,0
1082,22690595,19888166,Other driver error,2017-05-15,Auto,Yes,Injury only,33080.0,2.0,No,Male,1990-04-11,WI,712-651-9613,Gold,0
1083,12376894,11256802,Other driver error,2017-04-18,Auto,No,Material only,2840.0,4.0,No,Female,2064-10-22,LA,469-345-5617,Silver,0


In [17]:
cust_claim_data.isnull().sum()

claim_id               0
CUST_ID                0
incident_cause         0
claim_date             0
claim_area             0
police_report          0
claim_type             0
claim_amount           0
total_policy_claims    0
fraudulent             0
gender                 0
DateOfBirth            0
State                  0
Contact                0
Segment                0
alert_flag             0
dtype: int64

### 7. Calculate the age of customers in years. Based on the age, categorize the customers according to the below criteria
            Children < 18
            Youth 18-30
            Adult 30-60
            Senior > 60

In [18]:
cust_claim_data['DateOfBirth'] = np.where(pd.DatetimeIndex(cust_claim_data['DateOfBirth']).year < 2000, cust_claim_data.DateOfBirth, cust_claim_data.DateOfBirth - pd.offsets.DateOffset(years=100))
cust_claim_data

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment,alert_flag
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No,Female,1979-01-12,VT,789-916-8172,Platinum,0
1,33985796,75740424,Crime,2018-10-03,Home,Unknown,Material only,2980.0,3.0,No,Female,1970-01-13,ME,265-543-1264,Silver,1
2,53522022,30308357,Other driver error,2018-02-02,Auto,No,Material only,3369.5,1.0,Yes,Female,1984-03-11,TN,798-631-4758,Silver,0
3,63017412,30308357,Driver error,2018-04-04,Auto,No,Material only,1950.0,6.0,No,Female,1984-03-11,TN,798-631-4758,Silver,0
4,13015401,47830476,Natural causes,2018-06-17,Auto,No,Material only,1680.0,1.0,No,Female,1986-05-01,MA,413-187-7945,Silver,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,62020796,79539873,Other driver error,2018-09-06,Auto,Yes,Material only,1770.0,1.0,No,Female,1981-03-15,ND,459-425-4319,Platinum,0
1081,63269067,42364152,Other causes,2017-05-09,Home,No,Material only,2800.0,1.0,No,Female,1996-07-07,ID,529-462-1635,Silver,0
1082,22690595,19888166,Other driver error,2017-05-15,Auto,Yes,Injury only,33080.0,2.0,No,Male,1990-04-11,WI,712-651-9613,Gold,0
1083,12376894,11256802,Other driver error,2017-04-18,Auto,No,Material only,2840.0,4.0,No,Female,1964-10-22,LA,469-345-5617,Silver,0


In [19]:
def age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

In [20]:
cust_claim_data['Age']=cust_claim_data['DateOfBirth'].apply(age)
cust_claim_data.head(2) 

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment,alert_flag,Age
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No,Female,1979-01-12,VT,789-916-8172,Platinum,0,43
1,33985796,75740424,Crime,2018-10-03,Home,Unknown,Material only,2980.0,3.0,No,Female,1970-01-13,ME,265-543-1264,Silver,1,52


In [21]:
cust_claim_data.loc[(cust_claim_data.Age < 18), 'age_category'] = 'Children'
cust_claim_data.loc[(cust_claim_data.Age >= 18) & (cust_claim_data.Age < 30), 'age_category'] = 'Youth'
cust_claim_data.loc[(cust_claim_data.Age >= 30) & (cust_claim_data.Age < 60), 'age_category'] = 'Adult'
cust_claim_data.loc[(cust_claim_data.Age >= 60), 'age_category'] = 'Senior'
cust_claim_data

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment,alert_flag,Age,age_category
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No,Female,1979-01-12,VT,789-916-8172,Platinum,0,43,Adult
1,33985796,75740424,Crime,2018-10-03,Home,Unknown,Material only,2980.0,3.0,No,Female,1970-01-13,ME,265-543-1264,Silver,1,52,Adult
2,53522022,30308357,Other driver error,2018-02-02,Auto,No,Material only,3369.5,1.0,Yes,Female,1984-03-11,TN,798-631-4758,Silver,0,38,Adult
3,63017412,30308357,Driver error,2018-04-04,Auto,No,Material only,1950.0,6.0,No,Female,1984-03-11,TN,798-631-4758,Silver,0,38,Adult
4,13015401,47830476,Natural causes,2018-06-17,Auto,No,Material only,1680.0,1.0,No,Female,1986-05-01,MA,413-187-7945,Silver,0,36,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,62020796,79539873,Other driver error,2018-09-06,Auto,Yes,Material only,1770.0,1.0,No,Female,1981-03-15,ND,459-425-4319,Platinum,0,41,Adult
1081,63269067,42364152,Other causes,2017-05-09,Home,No,Material only,2800.0,1.0,No,Female,1996-07-07,ID,529-462-1635,Silver,0,25,Youth
1082,22690595,19888166,Other driver error,2017-05-15,Auto,Yes,Injury only,33080.0,2.0,No,Male,1990-04-11,WI,712-651-9613,Gold,0,32,Adult
1083,12376894,11256802,Other driver error,2017-04-18,Auto,No,Material only,2840.0,4.0,No,Female,1964-10-22,LA,469-345-5617,Silver,0,57,Adult


### 8. What is the average amount claimed by the customers from various segments? 

In [22]:
cust_claim_data.groupby('Segment').agg(avg_claim_amt=('claim_amount',np.mean))

,avg_claim_amt
Segment,
Gold,12829.631974
Platinum,12341.952267
Silver,12208.435570


### 9. What is the total claim amount based on incident cause for all the claims that have been done at least 20 days prior to 1st of October, 2018.

In [23]:
df = pd.DataFrame(cust_claim_data)
start_date = '2018-10-01'
end_date = pd.to_datetime('2018-10-01') - timedelta(days=20)
mask = (df['claim_date'] < start_date) & (df['claim_date'] >= end_date)
  
df = df.loc[mask]
df

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment,alert_flag,Age,age_category
7,39219616,19269962,Other causes,2018-09-26,Auto,No,Material only,2240.000000,2.0,No,Male,1977-05-13,NV,956-871-8691,Gold,0,45,Adult
17,53780662,21240703,Other driver error,2018-09-19,Auto,Unknown,Material only,1050.000000,1.0,No,Male,1963-07-22,MD,473-561-2675,Silver,1,58,Adult
70,43973221,83290720,Driver error,2018-09-26,Auto,Yes,Material and injury,12467.681373,1.0,No,Male,1966-12-06,VA,185-825-6547,Gold,0,55,Adult
118,65335959,61810954,Other driver error,2018-09-19,Auto,Unknown,Material and injury,36620.000000,1.0,No,Male,1969-05-18,TX,724-493-6179,Silver,1,53,Adult
134,4112041,58402401,Other causes,2018-09-12,Auto,Unknown,Material and injury,32710.000000,1.0,No,Male,1979-06-13,KS,134-129-8167,Silver,1,43,Adult
148,74738945,88449349,Other causes,2018-09-12,Auto,No,Material only,1080.000000,1.0,No,Male,1979-12-17,WA,975-547-5196,Platinum,0,42,Adult
162,3589979,14490971,Crime,2018-09-19,Auto,No,Material only,1880.000000,4.0,No,Female,1976-12-10,MN,614-652-1274,Silver,0,45,Adult
193,17871003,98905793,Other driver error,2018-09-19,Auto,Unknown,Material only,3139.500000,3.0,Yes,Female,1993-11-11,ID,823-482-5839,Platinum,1,28,Youth
224,5694962,9677945,Other causes,2018-09-26,Auto,No,Material only,1520.000000,1.0,No,Female,1987-05-28,SC,849-461-6419,Platinum,0,35,Adult
305,2581621,18213216,Other causes,2018-09-12,Auto,Unknown,Material and injury,21942.000000,2.0,Yes,Male,1969-10-06,KY,416-461-9145,Silver,1,52,Adult


In [24]:
df.groupby('incident_cause').agg(total_claim_amt=('claim_amount', np.sum))

,total_claim_amt
incident_cause,
Crime,6840.000000
Driver error,14597.681373
Natural causes,8418.000000
Other causes,247223.681373
Other driver error,47599.500000


### 10. How many adults from TX, DE and AK claimed insurance for driver related issues and causes?

In [25]:
group_driv= cust_claim_data[(cust_claim_data['incident_cause'] == 'Driver error') | (cust_claim_data['incident_cause'] == 'Other driver error')]
group_driv

,claim_id,CUST_ID,incident_cause,claim_date,claim_area,police_report,claim_type,claim_amount,total_policy_claims,fraudulent,gender,DateOfBirth,State,Contact,Segment,alert_flag,Age,age_category
0,54004764,21868593,Driver error,2017-11-27,Auto,No,Material only,2980.0,1.0,No,Female,1979-01-12,VT,789-916-8172,Platinum,0,43,Adult
2,53522022,30308357,Other driver error,2018-02-02,Auto,No,Material only,3369.5,1.0,Yes,Female,1984-03-11,TN,798-631-4758,Silver,0,38,Adult
3,63017412,30308357,Driver error,2018-04-04,Auto,No,Material only,1950.0,6.0,No,Female,1984-03-11,TN,798-631-4758,Silver,0,38,Adult
8,24050443,21831191,Other driver error,2017-10-04,Auto,No,Injury only,38306.5,3.0,Yes,Male,1977-05-17,NH,419-712-8513,Gold,0,45,Adult
9,91475471,21831191,Driver error,2018-02-09,Auto,Unknown,Material and injury,36810.0,1.0,No,Male,1977-05-17,NH,419-712-8513,Gold,1,45,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,28084004,82089910,Driver error,2018-06-03,Auto,No,Material only,2780.0,1.0,No,Male,1973-10-09,FL,257-248-4825,Silver,0,48,Adult
1078,47457836,6756905,Driver error,2017-01-21,Auto,No,Material only,2320.0,3.0,No,Male,1997-10-21,NE,924-427-8125,Silver,0,24,Youth
1080,62020796,79539873,Other driver error,2018-09-06,Auto,Yes,Material only,1770.0,1.0,No,Female,1981-03-15,ND,459-425-4319,Platinum,0,41,Adult
1082,22690595,19888166,Other driver error,2017-05-15,Auto,Yes,Injury only,33080.0,2.0,No,Male,1990-04-11,WI,712-651-9613,Gold,0,32,Adult


In [26]:
group_driv1=group_driv[group_driv['age_category']=='Adult'].groupby(['State']).size().reset_index().rename(columns={0:'Count of Adult'})
group_driv1[(group_driv1['State']== 'TX') | (group_driv1['State']== 'DE') | (group_driv1['State']== 'AK')]

,State,Count of Adult
0,AK,9
7,DE,10
42,TX,9


### 11. Draw a pie chart between the aggregated value of claim amount based on gender and segment. Represent the claim amount as a percentage on the pie chart.

In [27]:
agg_group=cust_claim_data.groupby(['Segment','gender']).agg(agg_claim_amt=('claim_amount', np.mean)).reset_index()
agg_group

,Segment,gender,agg_claim_amt
0,Gold,Female,12337.649897
1,Gold,Male,13242.028715
2,Platinum,Female,12277.117977
3,Platinum,Male,12415.993793
4,Silver,Female,11526.738532
5,Silver,Male,12823.080440


In [28]:
agg_group1= agg_group.loc[agg_group['Segment']=='Gold']
agg_group2= agg_group.loc[agg_group['Segment']=='Platinum']
agg_group3= agg_group.loc[agg_group['Segment']=='Silver']

In [29]:
labels = ["Female", "Male"]

fig = make_subplots(rows=1, cols=3, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])


fig.add_trace(go.Pie(
     values=agg_group1.agg_claim_amt ,
     labels=["Female", "Male"
             ],
     title="Gold",
     domain=dict(x=[0, 0.5]),
     name="Gold"), 
     row=1, col=1)

fig.add_trace(go.Pie(
     values=agg_group2.agg_claim_amt ,
     labels=["Female", "Male"
             ],
     title="Platinum",
     domain=dict(x=[0.5, 1.0]),
     name="Platinum"),
     row=1, col=2)

fig.add_trace(go.Pie(
     values=agg_group3.agg_claim_amt ,
     labels=["Female", "Male"
             ],
     title="Silver",
     domain=dict(x=[0.5, 1.0]),
     name="Silver"),
     row=1, col=3)


fig.update_layout(title_text="Aggregated value of claim amount based on gender and segment")

fig.show()

### 12. Among males and females, which gender had claimed the most for any type of driver related issues? E.g. This metric can be compared using a bar chart

In [30]:
group_gender= group_driv.iloc[:,[7, 10]]
group_gender

,claim_amount,gender
0,2980.0,Female
2,3369.5,Female
3,1950.0,Female
8,38306.5,Male
9,36810.0,Male
...,...,...
1075,2780.0,Male
1078,2320.0,Male
1080,1770.0,Female
1082,33080.0,Male


In [31]:
fig= px.bar(group_gender, x='gender', y='claim_amount', barmode='group', height=400)
fig.show()

### 13. Which age group had the maximum fraudulent policy claims? Visualize it on a bar chart.

In [32]:
fraud_age_group=cust_claim_data.groupby(['age_category','fraudulent']).size().reset_index().rename(columns={0:'Count of fraudlent'})
fraud_age_group

,age_category,fraudulent,Count of fraudlent
0,Adult,No,614
1,Adult,Yes,191
2,Senior,No,49
3,Senior,Yes,14
4,Youth,No,175
5,Youth,Yes,42


In [33]:
fig= px.bar(fraud_age_group, x='age_category', y='Count of fraudlent', color='fraudulent', barmode='group', height=400, title='Count of Fraudlent')
fig.show()

### 14. Visualize the monthly trend of the total amount that has been claimed by the customers. Ensure that on the “month” axis, the month is in a chronological order not alphabetical order.

In [34]:
cust_claim_data['month_year']= cust_claim_data['claim_date'].dt.to_period('M')

In [35]:
cust_claim_data['month_no']= cust_claim_data['claim_date'].dt.month
cust_claim_data['year']= cust_claim_data['claim_date'].dt.year

In [38]:
cust_claim_data_time_line1= cust_claim_data[['year', 'month_no', 'claim_amount']]
cust_claim_data_time_line1=cust_claim_data_time_line1.sort_values(['year','month_no'], ascending=[True, True])

In [39]:
cust_claim_data_time_line1

,year,month_no,claim_amount
67,2017,1,2910.000000
78,2017,1,12467.681373
90,2017,1,1180.000000
105,2017,1,37524.500000
120,2017,1,2967.000000
...,...,...,...
1049,2018,10,2270.000000
1054,2018,10,30890.000000
1055,2018,10,2420.000000
1074,2018,10,33330.000000


In [40]:
fig= px.bar(cust_claim_data_time_line1, x='month_no', y='claim_amount', color='year', barmode='group', height=400, title='monthly trend of the total amount that has been claimed by the customers')
fig.show()

### 15. What is the average claim amount for gender and age categories and suitably represent the above using a facetted bar chart, one facet that represents fraudulent claims and the other for non-fraudulent claims.

In [41]:
facet_fraud= cust_claim_data[['fraudulent', 'gender','age_category','month_no', 'claim_amount']]
facet_fraud=facet_fraud.groupby(['fraudulent','gender','age_category','month_no']).agg(agg_fraud=('claim_amount', np.mean)).reset_index()
facet_fraud

,fraudulent,gender,age_category,month_no,agg_fraud
0,No,Female,Adult,1,12452.136347
1,No,Female,Adult,2,8900.841970
2,No,Female,Adult,3,9881.922712
3,No,Female,Adult,4,12617.058824
4,No,Female,Adult,5,10824.834384
...,...,...,...,...,...
116,Yes,Male,Youth,8,22237.166667
117,Yes,Male,Youth,9,1932.000000
118,Yes,Male,Youth,10,6826.090686
119,Yes,Male,Youth,11,2468.666667


In [42]:
fig = px.bar(facet_fraud, x="age_category", y="agg_fraud", color="gender", barmode="group", facet_row="fraudulent")
fig.show()

## Based on the conclusions from exploratory analysis as well as suitable statistical tests, answer the below questions. Please include a detailed write-up on the parameters taken into consideration, the Hypothesis testing steps, conclusion from the p-values and the business implications of the statements.

### 16. Is there any similarity in the amount claimed by males and females?

### 1. Defining Hypothesis
Is there any similarity in the amount claimed by males and females

    H0: There is no difference in the amount claimed by males and females.
    H1: There is difference in the amount claimed by males and females.
    
The performance of the methods by using a 0.05 significance level. The hypothesis testing to check whether there is a difference between the amount claimed by males and females by using a 0.05 significance level.    

### 2. Assumption Check
    H₀: The data is normally distributed.
    H₁: The data is not normally distributed.

    H₀: The variances of the samples are the same.
    H₁: The variances of the samples are different. 

In [43]:
claim_male = cust_claim_data['claim_amount'].loc[cust_claim_data['gender']=="Male"]

claim_female = cust_claim_data['claim_amount'].loc[cust_claim_data['gender']=="Female"]

In [44]:
def check_normality(data):
    test_stat_normality, p_value_normality=stats.shapiro(data)
    print("p value:%.4f" % p_value_normality)
    if p_value_normality <0.05:
        print("Reject null hypothesis >> The data is not normally distributed")
    else:
        print("Fail to reject null hypothesis >> The data is normally distributed")  

In [45]:
def check_variance_homogeneity(group1, group2):
    test_stat_var, p_value_var= stats.levene(group1,group2)
    print("p value:%.4f" % p_value_var)
    if p_value_var <0.05:
        print("Reject null hypothesis >> The variances of the samples are different.")
    else:
        print("Fail to reject null hypothesis >> The variances of the samples are same.")

In [46]:
check_normality(claim_male)
check_normality(claim_female)

p value:0.0000
Reject null hypothesis >> The data is not normally distributed
p value:0.0000
Reject null hypothesis >> The data is not normally distributed


In [47]:
check_variance_homogeneity(claim_male, claim_female)

p value:0.3227
Fail to reject null hypothesis >> The variances of the samples are same.


### 3. Selecting the Proper Test

In [48]:
ttest,p_value = stats.ttest_ind(claim_male,claim_female)
print("p value:%.8f" % p_value)

if p_value <0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

p value:0.33036643
Fail to reject null hypothesis


### 4. Decision and Conclusion¶
we observe a p-value of 0.33036643 which is higher than significance level. This prove that there is no change in the amount claimed by males and females

### 17. Is there any relationship between age category and segment?

### 1. Defining Hypothesis

Conduct the hypothesis testing to check whether  Is there any relationship between age category and segment. So here are our hypotheses:

    H0: The mean of the samples is the same.
    H1: At least one of them is different.
    
The performance of the methods by using a 0.05 significance level. the hypothesis testing to check whether there is a difference between the performance of the methods by using a 0.05 significance level.    

In [49]:
cust_claim_data.columns

Index(['claim_id', 'CUST_ID', 'incident_cause', 'claim_date', 'claim_area',
       'police_report', 'claim_type', 'claim_amount', 'total_policy_claims',
       'fraudulent', 'gender', 'DateOfBirth', 'State', 'Contact', 'Segment',
       'alert_flag', 'Age', 'age_category', 'month_year', 'month_no', 'year'],
      dtype='object')

In [50]:
age_seg_data = pd.crosstab(cust_claim_data.age_category, cust_claim_data.Segment)
age_seg_data

Segment,Gold,Platinum,Silver
age_category,,,
Adult,279,270,256
Senior,24,21,18
Youth,72,71,74


### 2. Selecting the Proper Test

In [51]:
stats.chi2_contingency( age_seg_data )

(0.922002733439911,
 0.9213831643305384,
 4,
 array([[278.22580645, 268.58064516, 258.19354839],
        [ 21.77419355,  21.01935484,  20.20645161],
        [ 75.        ,  72.4       ,  69.6       ]]))

In [52]:
test_stat,p_value,dof, ex = stats.chi2_contingency( age_seg_data )
print("p value:%.17f" % p_value)
if p_value <0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

p value:0.92138316433053835
Fail to reject null hypothesis


### 3.Decision and Conclusion
Since the significance value of the test is greter than 0.05, we fail reject the null hypothesis. Therefore there is relationship between age category and segment

### 18. The current year has shown a significant rise in claim amounts as compared to 2016-17 fiscal average which was $10,000.

### 1. Defining Hypothesis

Conduct the hypothesis testing to check whether The current year has shown a significant rise in claim amounts as compared to 2016-17 fiscal average which was $10,000. So here are our hypothesis:

        H0: No rise in claim amounts as compared to 2016-17 fiscal average which was $10,000.
        H1: There is rise in claim amounts as compared to 2016-17 fiscal average which was $10,000.
    
We'll use our conventional α = 0.05.

In [53]:
year_compare =  cust_claim_data.groupby(["claim_date"])[["claim_amount"]].sum().reset_index()

In [54]:
#creating new columns which show "Month" and "Year"
year_compare['Monthly'] = year_compare['claim_date'].apply(lambda x:pd.Timestamp.strftime(x,format="%B"))
year_compare['Yearly'] = year_compare['claim_date'].apply(lambda x:pd.Timestamp.strftime(x,format="%Y"))

In [55]:
year_compare_2018 = year_compare.loc[ year_compare.Yearly == '2018', 'claim_amount' ]

### 2. Selecting the Proper Test

In [56]:
statsat, p_value = stats.ttest_1samp(year_compare_2018, 10000)
print("p value:%.6f" % p_value)
if p_value <0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

p value:0.000000
Reject null hypothesis


### 4. Decision and Conclusion
we observe a p-value very less.This confirms that there is rise in claim amounts as compared to 2016-17 fiscal average which was $10,000.

### 19. Is there any difference between age groups and insurance claims?

### 1. Defining Hypothesis

Conduct the hypothesis testing to check whether here any difference between age groups and insurance claims. So here are our hypotheses:

    H0: No relation between age groups and insurance claims.
    H1: There is relationship between age groups and insurance claims.
We'll use our conventional α = 0.05.

### 2. Assumption Check
    H₀: The data is normally distributed.
    H₁: The data is not normally distributed.

    H₀: The variances of the samples are the same.
    H₁: The variances of the samples are different.

In [57]:
cust_claim_data["total_policy_claims"] = cust_claim_data["total_policy_claims"].astype('float')

In [58]:
age_group_1 = cust_claim_data['total_policy_claims'].loc[cust_claim_data['age_category']=="Adult"]
age_group_2 = cust_claim_data['total_policy_claims'].loc[cust_claim_data['age_category']=="Youth"]
age_group_3 = cust_claim_data['total_policy_claims'].loc[cust_claim_data['age_category']=="Senior"]

In [59]:
check_normality(age_group_1)
check_normality(age_group_2)
check_normality(age_group_3)

p value:0.0000
Reject null hypothesis >> The data is not normally distributed
p value:0.0000
Reject null hypothesis >> The data is not normally distributed
p value:0.0000
Reject null hypothesis >> The data is not normally distributed


In [60]:
stat, pvalue_levene= stats.levene(age_group_1,age_group_2,age_group_3)

print("p value:%.4f" % pvalue_levene)
if pvalue_levene <0.05:
    print("Reject null hypothesis >> The variances of the samples are different.")
else:
    print("Fail to reject null hypothesis >> The variances of the samples are same.")

p value:0.5301
Fail to reject null hypothesis >> The variances of the samples are same.


### 3. Selecting the Proper Test

In [61]:
F, p_value = stats.f_oneway(age_group_1,age_group_2,age_group_3)
print("p value:%.6f" % p_value)
if p_value <0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

p value:0.530099
Fail to reject null hypothesis


### 4. Decision and Conclusion
The test result shows the pvalue is more than the significance level (0.05). We fail to reject null hypothesis which means no relation between age groups and insurance claims.

### 20. Is there any relationship between total number of policy claims and the claimed amount?

### 1. Defining Hypothesis
Conduct the hypothesis testing to check whether there any relationship between total number of policy claims and the claimed amount. So here are our hypotheses:

    H0: No relation between total number of policy claims and the claimed amount.
    H1: There is relationship between total number of policy claims and the claimed amount.
We'll use our conventional α = 0.05.

### 2. Assumption Check
    H₀: The data is normally distributed.
    H₁: The data is not normally distributed.

    H₀: The variances of the samples are the same.
    H₁: The variances of the samples are different.

In [62]:
cust_claim_data.columns

Index(['claim_id', 'CUST_ID', 'incident_cause', 'claim_date', 'claim_area',
       'police_report', 'claim_type', 'claim_amount', 'total_policy_claims',
       'fraudulent', 'gender', 'DateOfBirth', 'State', 'Contact', 'Segment',
       'alert_flag', 'Age', 'age_category', 'month_year', 'month_no', 'year'],
      dtype='object')

In [63]:
cust_claim_data.total_policy_claims.unique()

array([1., 3., 6., 2., 4., 5., 7., 8.])

In [64]:
claim_1 = cust_claim_data.loc[ cust_claim_data.total_policy_claims == 1.0, 'claim_amount' ]
claim_2 = cust_claim_data.loc[ cust_claim_data.total_policy_claims == 2.0, 'claim_amount' ]
claim_3 = cust_claim_data.loc[ cust_claim_data.total_policy_claims == 3.0, 'claim_amount' ]
claim_4 = cust_claim_data.loc[ cust_claim_data.total_policy_claims == 4.0, 'claim_amount' ]
claim_5 = cust_claim_data.loc[ cust_claim_data.total_policy_claims == 5.0, 'claim_amount' ]
claim_6 = cust_claim_data.loc[ cust_claim_data.total_policy_claims == 6.0, 'claim_amount' ]
claim_7 = cust_claim_data.loc[ cust_claim_data.total_policy_claims == 7.0, 'claim_amount' ]
claim_8 = cust_claim_data.loc[ cust_claim_data.total_policy_claims == 8.0, 'claim_amount' ]

In [65]:
check_normality(claim_1)
check_normality(claim_2)
check_normality(claim_3)
check_normality(claim_4)
check_normality(claim_5)
check_normality(claim_6)

p value:0.0000
Reject null hypothesis >> The data is not normally distributed
p value:0.0000
Reject null hypothesis >> The data is not normally distributed
p value:0.0000
Reject null hypothesis >> The data is not normally distributed
p value:0.0000
Reject null hypothesis >> The data is not normally distributed
p value:0.0000
Reject null hypothesis >> The data is not normally distributed
p value:0.1196
Fail to reject null hypothesis >> The data is normally distributed


In [66]:
stat, pvalue_levene= stats.levene(claim_1, claim_2, claim_3, claim_4, claim_5, claim_6, claim_7, claim_8)

print("p value:%.4f" % pvalue_levene)
if pvalue_levene <0.05:
    print("Reject null hypothesis >> The variances of the samples are different.")
else:
    print("Fail to reject null hypothesis >> The variances of the samples are same.")

p value:0.5703
Fail to reject null hypothesis >> The variances of the samples are same.


### 3. Selecting the Proper Test

In [67]:
F, p_value = stats.f_oneway(claim_1, claim_2, claim_3, claim_4, claim_5, claim_6, claim_7, claim_8)
print("p value:%.6f" % p_value)
if p_value <0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

p value:0.545526
Fail to reject null hypothesis


### 4. Decision and Conclusion
The test result shows the pvalue is more than the significance level (0.05). We fail to reject null hypothesis which means no relation between total number of policy claims and the claimed amount.